# Selenium for 법무법인_세종_뉴스레터

### Import Files

In [1]:
import requests
import lxml.html
import sqlite3 as sq3
from pandas.io import sql
import os
import re
import string
import pandas as pd
from tabulate import tabulate
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
import time
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.chrome.options import Options

## get_urls()
- gets the list of urls to detail pages

In [2]:
def get_urls():
    page = 0
    url_list = []
    
    response = requests.get('https://www.shinkim.com/kor/media/newsletter?page={}&size=10&keyword=&category=&biz-code=&member-code=&code=&gubun='.format(page))
    root = lxml.html.fromstring(response.content)
    
    #find the maximum number of pages
    max_page = int(root.xpath("//span[@class='total']")[0].text)
    
    while(max_page > page):
        response = requests.get('https://www.shinkim.com/kor/media/newsletter?page={}&size=10&keyword=&category=&biz-code=&member-code=&code=&gubun='.format(page))
        root = lxml.html.fromstring(response.content)
        time.sleep(0.5)
        
        li = root.xpath('//li[@class="post-item"]')
        for each in li:
            a = each.xpath('div/div/div/p/a')[0]
            url = 'https://www.shinkim.com' + a.get('href')
            url_list.append(url)

        if (len(url_list) >=0 & len(url_list)%10 == 0):
            page += 1

    return url_list

## remove_punc()
- removes punctuations from string

In [3]:
def remove_punc(data):

    punc = '[!"#$%&\'()*+,-./:;<=>?[\]^_`{|}~“”·「」△《》•‘’○※▷①②③④【】『∙』·-“”▲I]'
    new_string = re.sub(punc, '', data) # 특수문자 제거
    new_string2 = re.sub('\n', ' ', new_string) # newline 제거
    new_string3 = re.sub('\\s+', ' ', new_string2) # multiple spaces 제거
    return new_string3

## db_save()
- saves data as db file

In [4]:
def db_save(ARTICLE_LIST, db_name, table_name):
    with sq3.connect(os.path.join('.',db_name)) as con: # sqlite DB 파일이 존재하지 않는 경우 파일생성
        try:
            ARTICLE_LIST.to_sql(name = table_name, con = con, index = False, if_exists='replace') 
            #if_exists : {'fail', 'replace', 'append'} default : fail
        except Exception as e:
            print(str(e))
        print(len(ARTICLE_LIST), '건 저장완료..')

## db_select()
- read data from db file

In [5]:
def db_select(db_name, table_name):
    with sq3.connect(db_name) as con: 
        try:
            query = 'SELECT * FROM {}'.format(table_name)
            df = pd.read_sql(query, con = con)
        except Exception as e:
            print(str(e)) 
        return df

In [6]:
def db_delete(db_name, table_name):
    with sq3.connect(db_name) as con: 
        try:
            cur = con.cursor()
            sql = 'DELETE FROM {}'.format(table_name)
            cur.execute(sql)
        except Exception as e:
            print(str(e))

## get_info()
- returns detailed information on the article pages as list

In [7]:
def get_info(url):
    details = []
    articles = []
    articleString = ""
    
    driver.get(url)
    title = driver.find_element('xpath','//*[@id="content"]/div[2]/div[1]/div[1]/div[1]/h4').text
    date = driver.find_element('xpath','//*[@id="content"]/div[2]/div[1]/div[1]/div[2]/span').text
    topic = driver.find_element('xpath','//*[@id="content"]/div[2]/div[1]/div[1]/div[1]/span').text
    article = driver.find_elements('xpath','//*[@class="post-content main-text"]/p')
    
    for each in article:
        articles.append(each.text)
    articleString = ' '.join(articles)
    
    driver.implicitly_wait(5)
    
    words = remove_punc(articleString)
    details.append(title)
    details.append(date)
    details.append(topic)
    details.append(words)
    
    return details

## db_save_as_csv()
- saves dataframe as csv

In [8]:
def db_save_as_csv(data):
    data.to_csv("법무법인_세종_뉴스레터.csv", index=False, encoding='utf-8')

### ---------------------------------------------------------------------------------------------------------------------
# Main
This is the main part which uses functions to read articles and saves the data as csv file
### ---------------------------------------------------------------------------------------------------------------------

In [9]:
urls = get_urls()
df_list = []
print("There are ",len(urls)," urls in list.")

options = webdriver.ChromeOptions() 
#options.add_argument("--start-maximized");  # Chrome 브라우저 최대화 설정
options.add_argument('--headless')  # headless 모드
options.add_argument('--disable-gpu')  # 그래픽 가속 해제 (오류 방지)
#options.add_argument('--mute-audio')  # 음소거모드 적용

#driver = webdriver.Chrome('service = Service(ChromeDriverManager().install())', options = options)
driver = webdriver.Chrome('C:/Users/user/.wdm/drivers/chromedriver/win32/105.0.5195/chromedriver.exe', chrome_options = options) #드라이버경로 지정

for url in urls:
    details = get_info(url)
    
    df = pd.DataFrame([{
        "제목": details[0],
        "날짜": details[1],
        "주제": details[2],
        "본문": details[3]
    }])
    
    df_list.append(df)
    driver.implicitly_wait(4)

ARTICLE_LIST = pd.concat(df_list)

db_save(ARTICLE_LIST, '법무법인_세종_뉴스레터.db', '뉴스레터')
db_save_as_csv(ARTICLE_LIST)

driver.close()
driver.quit()

There are  910  urls in list.


C:\Users\user\AppData\Local\Temp\ipykernel_16176\899453363.py:12: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('C:/Users/user/.wdm/drivers/chromedriver/win32/105.0.5195/chromedriver.exe', chrome_options = options) #드라이버경로 지정
C:\Users\user\AppData\Local\Temp\ipykernel_16176\899453363.py:12: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome('C:/Users/user/.wdm/drivers/chromedriver/win32/105.0.5195/chromedriver.exe', chrome_options = options) #드라이버경로 지정


910 건 저장완료..


In [10]:
df1 = db_select('법무법인_세종_뉴스레터.db', '뉴스레터')
df1

,제목,날짜,주제,본문
0,Policy and Law Report_Vol.153 – 정부 주요 정책 및 입법정...,2022.09.20,입법전략자문,법무법인유 세종 입법전략자문팀Legislative and Government Aff...
1,순환자원으로 인정받을 수 있는 폐기물의 범위가 확대됩니다 - ‘자원순환기본법 시행령...,2022.09.16,환경,환경부는 순환자원 인정 요건 완화를 골자로 하는 자원순환기본법 시행령 일부개정안이하...
2,상속세 및 증여세법상 성실공익법인의 요건으로 전용계좌의 신고의무는 포함되지 않는다는...,2022.09.13,조세,최근 서울고등법원은 성실공익법인의 다른 요건을 갖춘 공익법인이 전용계좌를 개설하여 ...
3,검사의 수사개시 범위 관련 개정 대통령령(2022.9.10.자)의 주요 내용,2022.09.13,형사,2022 9 10 검사의 직접 수사개시 범위를 축소시키는 내용 등이 포함된 개정 형...
4,Policy and Law Report_Vol.152 – 정부 주요 정책 및 입법정...,2022.09.13,입법전략자문,법무법인유 세종 입법전략자문팀Legislative and Government Aff...
...,...,...,...,...
905,뉴스레터 Vol 65 | 2011,2011.02,General,뉴스레터 Vol 65 2011 자세한 내용은 우측 상단에 있는 PDF 파일을 다운로...
906,소득세법상 특수관계자간 상장주식 양도시 시가,2011.02,조세,2011년 1월 13일 대법원에서는 소득세법상 상장주식의 시가의 산정과 관련하여 중...
907,“미네르바”법 위헌결정과 기업법무에서의 시사점,2011.02,General,2008년 7월 글로벌 금융위기가 발생하였을 때 미네르바라는 필명의 한 인터넷 논객...
908,소위 ‘물량 몰아주기’에 관한 공정거래위원회의 심사기준 개정,2011.01,공정거래,공정거래위원회는 2010 12 22 부당한 지원행위 심사지침을 개정2011 1 1 ...
